In [1]:
from __future__ import division
from full_prod_DRA import *
from buchi import buchi_from_ltl
import numpy as np
from env_sensing_error import *
import scipy
# from plot_path_for_prod import *
from graphviz import Source
from qnetwork import *
from utils import *
import matplotlib.pyplot as plt
# from Plot_Path import *
import tensorflow as tf
import sys
from dra_planning import *
import time

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def train(sess, env, qnet, prod_planner):
    
    global EXPLORATION_RATE
    global GUIDE_RATE
  
    summary_ops, summary_vars = build_summaries()
    if not RESTORE:
        sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    
    qnet.update_target()
    
    replay_buffer = ReplayBuffer(BUFFER_SIZE, RANDOM_SEED)
    
    while len(prod_planner.opt_path) == 0:
        env.step(np.random.randint(0,qnet.action_dim))
        prod_planner.update_wfts_ap()
        env.update_dynamic_rabin()
        prod_planner.get_global_opt()
    print "Global Solution Found"
    prod_planner.get_opt_rabin()
    
    saved_dra_planners = {}
    
    for num_epi in range(MAX_EPISODES):
        
#         print "Epi: ", num_epi

        s = env.reset()
        s = list(np.unravel_index(s, env.shape))
#         prod_planner.replace_region_list()
        prod_planner.update_wfts_ap()

        ep_reward = 0
        ep_ave_max_q = 0
        
        reward_list = []
        
        train_time = 0
        batch_time = 0
        gym_time = 0
        guide_time = 0

        for j in range(MAX_EPISODE_LEN):
            
#             print "Step: ", j
            
            gym_start = time.time()

            rand_num = np.random.rand(1)
    
            if rand_num <= EXPLORATION_RATE:
                a = np.random.randint(0,qnet.action_dim)
                s2, r, terminal, info = env.step(a)
                
            elif rand_num <= GUIDE_RATE+EXPLORATION_RATE and rand_num > EXPLORATION_RATE:
#                 print "GUIDE"
                guide_start = time.time()
                
                if rand_num > EXPLORATION_RATE + 0.9*GUIDE_RATE:
                # Only update global plan with 0.2 prob for efficiency
                    env.update_dynamic_rabin()
                    prod_planner.get_global_opt()
                    prod_planner.get_opt_rabin()
                    
                if len(prod_planner.opt_path) > 0:
#                     print "S: ", s
#                     print "last a: ", a
                    new_ltl = prod_planner.get_next_ltl(s[-1])
                    
#                     if new_ltl in saved_dra_planners.keys():
                    guide_path = prod_planner.get_local_opt(s[:-1], new_ltl)
                    if guide_path != None:
#                         print "GUIDE"
                        a = convert_path_to_action(guide_path)
                    else:
                        a = np.random.randint(0,qnet.action_dim)
#                     else:
#                         guide_path = prod_planner.get_local_opt(s[:-1], new_ltl)
#                         saved_dra_planners[new_ltl] = prod_planner.dra_full_prod
#                         if guide_path != None:
#                             a = convert_path_to_action(guide_path)
#                         else:
#                             a = np.random.randint(0,qnet.action_dim)
                        
                else:
                    a = np.random.randint(0,qnet.action_dim)
                
                s2, r, terminal, info = env.step(a)
                
                guide_time += time.time() - guide_start
                
            else:
                a = np.argmax(qnet.predict_q(np.reshape(s, (1, qnet.state_dim))))
                s2, r, terminal, info = env.step(a)

            prod_planner.update_wfts_ap()
                
            gym_time += time.time() - gym_start
            
            batch_start = time.time()
            
            s2 = list(np.unravel_index(s2, env.shape))

            replay_buffer.add(np.reshape(s, (qnet.state_dim,)), np.reshape(a, (1,)), r,
                              terminal, np.reshape(s2, (qnet.state_dim,)))
            batch_time += time.time() - batch_start

            # Keep adding experience to the memory until
            # there are at least minibatch size samples
            if replay_buffer.size() > MINIBATCH_SIZE:
                
                batch_start = time.time()
                s_batch, a_batch, r_batch, t_batch, s2_batch = replay_buffer.sample_batch(MINIBATCH_SIZE)
#                 print "sbatch: ", s_batch
                # Calculate targets
                target_q = qnet.predect_target(s2_batch)

                y_i = []
                for k in range(MINIBATCH_SIZE):
                    if t_batch[k]:
                        y_i.append(r_batch[k])
                    else:
                        y_i.append(r_batch[k] + GAMMA * np.amax(target_q[k]))
                        
                batch_time += time.time() - batch_start

                # Update the critic given the targets
                train_start = time.time()
                predicted_q_value, _ = qnet.train(s_batch, a_batch, np.reshape(y_i, (MINIBATCH_SIZE, 1)), num_epi)

                ep_ave_max_q += np.amax(predicted_q_value)
                
                # Update target networks
                qnet.update_target()

                train_time += time.time() - train_start

            s = s2
            ep_reward += r

            if terminal or j == MAX_EPISODE_LEN-1:
                
                if EXPLORATION_RATE > 0.02 and terminal:
                    EXPLORATION_RATE = EXPLORATION_RATE*0.98
                    GUIDE_RATE = GUIDE_RATE*0.98
                    
                reward_list += [ep_reward]
                
                if np.average(reward_list[-10:]) > LR_DECAY_TRUNCATION:
                    qnet.decay_learning_rate(0.98)

                print('| Reward: {:d} | Episode: {:d} | Qmax: {:.4f} | Exploration: {:.6f} | Step: {:d} | LR: {:.8f}'.format(int(ep_reward), \
                        num_epi, (ep_ave_max_q / float(j)), EXPLORATION_RATE, j, qnet.get_learning_rate()))
                
                f = open("stats/" + file_appendix + "_stats.txt", "ab")
                f.write("| Reward: " + str(int(ep_reward)) 
                        +" | Episode: " + str(num_epi) 
                        + " | Qmax: " + str(ep_ave_max_q / float(j)) 
                        + " | Exploration: " + str(EXPLORATION_RATE)
                        + " | Step: " + str(j)
                        + " | LR:" + str(qnet.get_learning_rate()) + "\n")
                f.close()
                
                f = open("stats/" + file_appendix + "_stats_time.txt", "ab")
                f.write(" | Episode: " + str(num_epi) 
                        + " | Train: " + str(train_time) 
                        + " | Gym: " + str(gym_time)
                        + " | Batch: " + str(batch_time) 
                        + " | Guide: " + str(guide_time)
                        + "\n")
                f.close()
                
                break
                

In [3]:
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B"
# LTL = "[] (p1 -> !(X p1) U (p2 || p3) ) && []<>p1"
# LTL = "T && []<>A && []<>B"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C && []!D"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C"
# LTL = "<>(A && <>(B && <> T))"
# LTL = "<>(A && <>B) && <>[]T && []!C"
LTL = "<>(A && <>T) && []!C"
# LTL = "<>(A && <>(B && <>T)) && []<>(A||T) && []<>B && []!C"
# LTL = "<>(A && <>(B && <>T)) && []!C"
# LTL = "<>(A && <>D) && <>(B && <>E) && []<>T && []<>(D || E) && []!C"

LEARNING_RATE = 0.0015
GAMMA = 0.99
# GAMMA = 0.7
TAU = 0.001
BUFFER_SIZE = 10**6
MINIBATCH_SIZE = 64
RANDOM_SEED = 210
MAX_EPISODES = 50000
MAX_EPISODE_LEN = 500
file_appendix = "Guide_Planning_" + time.ctime()[4:16].replace("  ","").replace(" ","_").replace(":","-") + "_large_" + LTL
# file_appendix = "Feb8_03-47"
SUMMARY_DIR = './results/tf_ddqn_' + file_appendix
SAVE_DIR = "./saved_model/" + file_appendix + "/ddqn.ckpt"
EXPLORATION_RATE = 0.2
GUIDE_RATE = 0.6
LR_DECAY_TRUNCATION = -200
RESTORE = 0
if sys.platform == "darwin":
    DEVICE = "/device:CPU:0"
else:
    DEVICE = "/device:GPU:0"

In [4]:
env = CurrentWorld(LTL)
prod_planner = Prod_Planning(env, LTL)
# with open("my.dot", "r") as dotfile:
#     text = dotfile.read()
# Source(text)

In [ ]:
config=tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
       
    state_dim = 3
    action_dim = 5
    
    if RESTORE:
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)
        Qnet.saver.restore(sess, RESTORE_PATH)
        train(sess, env, Qnet, prod_planner)
        
    else:
        np.random.seed(RANDOM_SEED)
        tf.set_random_seed(RANDOM_SEED)
        env.seed(RANDOM_SEED)
    
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)

        train(sess, env, Qnet, prod_planner)

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Global Solution Found
| Reward: -720 | Episode: 0 | Qmax: 0.0000 | Exploration: 0.196000 | Step: 55 | LR: 0.00150000
| Reward: -3150 | Episode: 1 | Qmax: 17.2250 | Exploration: 0.192080 | Step: 145 | LR: 0.00150000
| Reward: -1640 | Episode: 2 | Qmax: 15.0672 | Exploration: 0.188238 | Step: 138 | LR: 0.00150000
| Reward: -740 | Episode: 3 | Qmax: 28.6989 | Exploration: 0.184474 | Step: 39 | LR: 0.00150000
| Reward: -480 | Episode: 4 | Qmax: 42.4368 | Exploration: 0.180784 | Step: 31 | LR: 0.00150000
| Reward: -320 | Episode: 5 | Qmax: 49.4814 | Exploration: 0.177168 | Step: 33 | LR: 0.00150000
| Reward: -1080 | Episode: 6 | Qmax: 61.3744 | Exploration: 0.173625 | Step: 73 | LR: 0.00150000
| Reward: -680 | Episode: 7 | Qmax: 76.8072 | Exploration: 0.170153 | Step: 69 | LR: 0.00150000
| Reward: -290 | Episode: 8 | Qmax: 73.7530 | Exploration: 0.166750 | Step: 30 | LR: 0.00150000
| Reward: -950 | Episode: 9 | Qmax: 77.9628 | Exploration: 0.163415 | Step: 87 | LR: 0.00150000
| Reward: -420

| Reward: -140 | Episode: 84 | Qmax: 95.0168 | Exploration: 0.044850 | Step: 24 | LR: 0.00138355
| Reward: -250 | Episode: 85 | Qmax: 86.5527 | Exploration: 0.043953 | Step: 26 | LR: 0.00138355
| Reward: -130 | Episode: 86 | Qmax: 100.4584 | Exploration: 0.043074 | Step: 23 | LR: 0.00135588
| Reward: -360 | Episode: 87 | Qmax: 82.5439 | Exploration: 0.042212 | Step: 46 | LR: 0.00135588
| Reward: -750 | Episode: 88 | Qmax: 80.0972 | Exploration: 0.041368 | Step: 40 | LR: 0.00135588
| Reward: -280 | Episode: 89 | Qmax: 83.3066 | Exploration: 0.040541 | Step: 38 | LR: 0.00135588
| Reward: -230 | Episode: 90 | Qmax: 94.6116 | Exploration: 0.039730 | Step: 24 | LR: 0.00135588
| Reward: -230 | Episode: 91 | Qmax: 85.4592 | Exploration: 0.038935 | Step: 33 | LR: 0.00135588
| Reward: -190 | Episode: 92 | Qmax: 91.1822 | Exploration: 0.038156 | Step: 29 | LR: 0.00132876
| Reward: -410 | Episode: 93 | Qmax: 88.9623 | Exploration: 0.037393 | Step: 42 | LR: 0.00132876
| Reward: -160 | Episode: 94 

| Reward: -110 | Episode: 168 | Qmax: 90.6695 | Exploration: 0.019990 | Step: 21 | LR: 0.00066855
| Reward: -110 | Episode: 169 | Qmax: 97.6399 | Exploration: 0.019990 | Step: 21 | LR: 0.00065518
| Reward: -120 | Episode: 170 | Qmax: 91.3473 | Exploration: 0.019990 | Step: 22 | LR: 0.00064208
| Reward: -100 | Episode: 171 | Qmax: 101.5931 | Exploration: 0.019990 | Step: 20 | LR: 0.00062923
| Reward: -350 | Episode: 172 | Qmax: 93.2698 | Exploration: 0.019990 | Step: 45 | LR: 0.00062923
| Reward: -200 | Episode: 173 | Qmax: 98.0054 | Exploration: 0.019990 | Step: 30 | LR: 0.00062923
| Reward: -110 | Episode: 174 | Qmax: 92.5435 | Exploration: 0.019990 | Step: 21 | LR: 0.00061665
| Reward: -130 | Episode: 175 | Qmax: 96.6543 | Exploration: 0.019990 | Step: 23 | LR: 0.00060432
| Reward: -320 | Episode: 176 | Qmax: 97.5905 | Exploration: 0.019990 | Step: 24 | LR: 0.00060432
| Reward: -110 | Episode: 177 | Qmax: 99.2858 | Exploration: 0.019990 | Step: 21 | LR: 0.00059223
| Reward: -180 | Ep

| Reward: -150 | Episode: 252 | Qmax: 96.2409 | Exploration: 0.019990 | Step: 25 | LR: 0.00014692
| Reward: -100 | Episode: 253 | Qmax: 101.6162 | Exploration: 0.019990 | Step: 20 | LR: 0.00014399
| Reward: -90 | Episode: 254 | Qmax: 98.4339 | Exploration: 0.019990 | Step: 19 | LR: 0.00014111
| Reward: -90 | Episode: 255 | Qmax: 100.8452 | Exploration: 0.019990 | Step: 19 | LR: 0.00013828
| Reward: -90 | Episode: 256 | Qmax: 103.6123 | Exploration: 0.019990 | Step: 19 | LR: 0.00013552
| Reward: -110 | Episode: 257 | Qmax: 107.4151 | Exploration: 0.019990 | Step: 21 | LR: 0.00013281
| Reward: -90 | Episode: 258 | Qmax: 98.3238 | Exploration: 0.019990 | Step: 19 | LR: 0.00013015
| Reward: -180 | Episode: 259 | Qmax: 103.6843 | Exploration: 0.019990 | Step: 19 | LR: 0.00012755
DDQN Saved
| Reward: -90 | Episode: 260 | Qmax: 105.7855 | Exploration: 0.019990 | Step: 19 | LR: 0.00012500
| Reward: -90 | Episode: 261 | Qmax: 106.2875 | Exploration: 0.019990 | Step: 19 | LR: 0.00012250
| Reward

...

| Reward: -70 | Episode: 2414 | Qmax: 105.9743 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -90 | Episode: 2415 | Qmax: 105.4513 | Exploration: 0.019990 | Step: 19 | LR: 0.00000000
| Reward: -70 | Episode: 2416 | Qmax: 106.5375 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -90 | Episode: 2417 | Qmax: 105.7870 | Exploration: 0.019990 | Step: 19 | LR: 0.00000000
| Reward: -80 | Episode: 2418 | Qmax: 106.3174 | Exploration: 0.019990 | Step: 18 | LR: 0.00000000
| Reward: -70 | Episode: 2419 | Qmax: 106.1140 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
DDQN Saved
| Reward: -70 | Episode: 2420 | Qmax: 104.5081 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -200 | Episode: 2421 | Qmax: 104.0460 | Exploration: 0.019990 | Step: 21 | LR: 0.00000000
| Reward: -70 | Episode: 2422 | Qmax: 104.9257 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2423 | Qmax: 105.8408 | Exploration: 0.019990 | Step: 17 | LR: 0.0000000

| Reward: -70 | Episode: 2497 | Qmax: 107.4368 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -90 | Episode: 2498 | Qmax: 106.0234 | Exploration: 0.019990 | Step: 19 | LR: 0.00000000
| Reward: -90 | Episode: 2499 | Qmax: 106.6854 | Exploration: 0.019990 | Step: 19 | LR: 0.00000000
DDQN Saved
| Reward: -70 | Episode: 2500 | Qmax: 107.2911 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2501 | Qmax: 107.8414 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2502 | Qmax: 106.8620 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2503 | Qmax: 107.2856 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2504 | Qmax: 107.1232 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -110 | Episode: 2505 | Qmax: 105.1201 | Exploration: 0.019990 | Step: 21 | LR: 0.00000000
| Reward: -70 | Episode: 2506 | Qmax: 106.4034 | Exploration: 0.019990 | Step: 17 | LR: 0.0000000

DDQN Saved
| Reward: -70 | Episode: 2580 | Qmax: 105.6372 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2581 | Qmax: 105.1579 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -80 | Episode: 2582 | Qmax: 106.4727 | Exploration: 0.019990 | Step: 18 | LR: 0.00000000
| Reward: -70 | Episode: 2583 | Qmax: 106.4326 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2584 | Qmax: 106.5197 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2585 | Qmax: 106.2345 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2586 | Qmax: 104.4528 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2587 | Qmax: 105.2397 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -90 | Episode: 2588 | Qmax: 103.7495 | Exploration: 0.019990 | Step: 19 | LR: 0.00000000
| Reward: -70 | Episode: 2589 | Qmax: 105.5967 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000

| Reward: -70 | Episode: 2663 | Qmax: 104.9507 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -170 | Episode: 2664 | Qmax: 103.6131 | Exploration: 0.019990 | Step: 18 | LR: 0.00000000
| Reward: -140 | Episode: 2665 | Qmax: 105.7685 | Exploration: 0.019990 | Step: 15 | LR: 0.00000000
| Reward: -70 | Episode: 2666 | Qmax: 106.8747 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -110 | Episode: 2667 | Qmax: 105.5006 | Exploration: 0.019990 | Step: 21 | LR: 0.00000000
| Reward: -80 | Episode: 2668 | Qmax: 105.7233 | Exploration: 0.019990 | Step: 18 | LR: 0.00000000
| Reward: -70 | Episode: 2669 | Qmax: 108.1396 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2670 | Qmax: 106.6970 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -70 | Episode: 2671 | Qmax: 109.1233 | Exploration: 0.019990 | Step: 17 | LR: 0.00000000
| Reward: -110 | Episode: 2672 | Qmax: 105.7734 | Exploration: 0.019990 | Step: 21 | LR: 0.00000000
| Rewa

KeyboardInterrupt: 

In [ ]:
env.reset()

In [ ]:
env.s

In [ ]:
np.where(np.array(prod_planner.opt_rabin) == 1)[0][0]

In [ ]:
np.unravel_index(env.s,(10,10,5))

In [ ]:
np.unravel_index(env.s,(10,10,5))

In [ ]:
ns,r,_,_ = env.step(1)
print np.unravel_index(ns,(10,10,5)),r

In [ ]:
prod_planner.get_global_opt()

In [ ]:
env.ap_dict

In [ ]:
env.last_ap_dict

In [ ]:
env.last_dynamic_coord_dict[(2,2)]

In [ ]:
env.rabin.coord_dict[(2,2)]

In [ ]:
env.dynamic_coord_dict[(2,2)]

In [ ]:
env.rabin.possible_states(1)

In [ ]:
[env.rabin.graph[str(1)][str(0)][k]["label"] for k in range(len(env.rabin.graph[str(1)][str(0)]))]

In [ ]:
env.rabin.check_ap("C", u' !A&!T&C')

In [ ]:
env.rabin.graph["0"]["0"]

In [ ]:
env._calculate_transition_prob((4, 3, 1), [0,1])

In [ ]:
np.unravel_index(221,(10,10,5))

In [ ]:
env.shape

In [ ]:
current = (4,3,1)
delta=[0,1]
delta_list = [[-1, 0], [1, 0], [0, -1], [0, 1], [0, 0]]

In [ ]:
new_position_candidates = [np.array(current[:2]) + np.array(delta)]
new_position_candidates += [np.array(current[:2]) + np.array(i) for i in delta_list if i != delta]

In [ ]:
print new_positions

In [ ]:
new_positions = [env._limit_coordinates(i).astype(int) for i in new_position_candidates]

In [ ]:
next_rabin_state = [env.rabin.next_state(current, tuple(i)) for i in new_positions]

In [ ]:
env.rabin.next_state((4,3,1),(4,4))

In [ ]:
env.rabin.coord_dict[(2,7)]

In [ ]:
env.rabin.next_state((4,3,1),[0,1])

In [ ]:
env.rabin.deadlock

In [ ]:
np.where(np.array(prod_planner.opt_rabin) == 8)[0]

In [ ]:
env = CurrentWorld(LTL.lower())

In [ ]:
prod_planner = Prod_Planning(env, LTL.lower())

In [ ]:
prod_planner.update_wfts_ap()

In [ ]:
env.dynamic_coord_dict[(5,5)]

In [ ]:
env.last_dynamic_coord_dict[(5,5)]

In [ ]:
prod_planner.region_list[np.ravel_multi_index((5,5),(10,10))].ap

In [ ]:
env.ap_dict

In [ ]:
env.last_ap_dict

In [ ]:
env.step(1)